In [1]:
import requests
import lxml.html as lh
import pandas as pd
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
#getting url, apparently this one only works with the urls with a year at the back...
url='https://www.salary.com/tools/executive-compensation-calculator/accenture-plc-executive-salaries?year=2009'
#year = int(url[-4:])
#using Beautiful Soup to read the contents of the website
uClient = uReq(url)
page_html = uClient.read()
uClient.close()

In [3]:
#html parser
page_soup = soup(page_html, "html.parser")

In [4]:
#grabs the divs that contain all the names, positions, and salaries
containers = page_soup.findAll("div", {"class":"border-bottom font-semibold"})

In [5]:
#see how many rows we have to go through
len(containers)

5

In [6]:
#Makes a list of to make the graph later on
col=[('CEO Name', ['']), ('CEO Pay', [0]), ('Year', [0])]

In [7]:
#this will find the data for the ceo by going through the relevant rows
for i in range(len(containers)):
    #this is to find the name of the person in the row
    name_container = containers[i].div.a.text
    #this is to find the position the person held
    pos_container = containers[i].find("div", "padding-bottom10 text-size15")
    #this is to find their salary
    sal = containers[i].find_all('div',class_='padding-left15')[1].div.text
    #this part converts the salary to a number
    sal = sal.strip()
    sal = sal.replace(r'$', '')
    sal = sal.replace(r',', '')
    try:
        sal=int(sal)
    except:
        print(sal)
        print('the ceo pay is not a number')
        break
    try:
        #this tries to verify that the person in the row is the ceo
        if "CEO" in pos_container.text or "Chief Executive Officer" in pos_container.text and not ("Former" in pos_container.text or "former" in pos_container.text):
            #If there are multiple CEOs, I'll take the one with the highest salary
            if sal > col[1][1][0]:
                col[0][1][0] = (name_container)
                col[1][1][0] = (sal)
                col[2][1][0] = (year)
    except:
        pass


In [8]:
#making the dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [9]:
#displaying the dataframe
df

,CEO Name,CEO Pay,Year
0,William D. Green,18277099,0


In [10]:
#the following were for testing purposes
container = containers[1]
print(container.div.a.text)
container = containers[1].find("div", "padding-bottom10 text-size15")
print(container.text)
"CEO" in container.text
c = containers[0]
sal = c.find_all('div',class_='padding-left15')[1].div.text
sal = sal.strip()
sal = sal.replace(r'$', '')
sal = sal.replace(r',', '')
try:
    sal=int(sal)
except:
    print('salary isn\'t a number')
sal

William D. Green
Chief Executive Officer


6678758